In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,BaggingClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

###### df=pd.read_csv("/kaggle/input/human-stress-prediction/Stress.csv")
df=df.drop(columns=["post_id","sentence_range"])

In [4]:
df.sample()

,subreddit,text,label,confidence,social_timestamp
2398,relationships,Generally good marriage. We can both be stubbo...,0,0.6,1515184399


In [6]:
df.nunique()

subreddit             10
text                2820
label                  2
confidence            10
social_timestamp    2343
dtype: int64

In [5]:
df

,subreddit,text,label,confidence,social_timestamp
0,ptsd,"He said he had not felt that way before, sugge...",1,0.800000,1521614353
1,assistance,"Hey there r/assistance, Not sure if this is th...",0,1.000000,1527009817
2,ptsd,My mom then hit me with the newspaper and it s...,1,0.800000,1535935605
3,relationships,"until i met my new boyfriend, he is amazing, h...",1,0.600000,1516429555
4,survivorsofabuse,October is Domestic Violence Awareness Month a...,1,0.800000,1539809005
...,...,...,...,...,...
2833,relationships,"* Her, a week ago: Precious, how are you? (I i...",0,1.000000,1515187044
2834,ptsd,I don't have the ability to cope with it anymo...,1,1.000000,1539827412
2835,anxiety,In case this is the first time you're reading ...,0,1.000000,1539269312
2836,almosthomeless,Do you find this normal? They have a good rela...,0,0.571429,1488938143


In [14]:
def text_preprocessing(text):
    data=text.lower()
    data_re=re.sub(r'\W+',' ',data)
    data_token=word_tokenize(data_re)
    tokens=[word for word in data_token if word not in stopwords.words('english')]
    return " ".join(tokens)

In [15]:
df["text"]=df["text"].apply(text_preprocessing)

In [17]:
df.isnull().sum()


subreddit           0
text                0
label               0
confidence          0
social_timestamp    0
dtype: int64

In [31]:
tfidf=TfidfVectorizer()
tfidf_data=tfidf.fit_transform(df["text"])
tfidf_data=tfidf_data.toarray()
tfidf_data=pd.DataFrame(tfidf_data)

In [22]:
one_hot=OneHotEncoder(sparse_output=False)
one=one_hot.fit_transform(df[["subreddit"]])
one=pd.DataFrame(one,columns=one_hot.get_feature_names_out())

In [45]:
new_df=pd.concat([df,one,tfidf_data],axis=1)
new_df=new_df.drop(columns=["subreddit","text","label"])
new_df

,confidence,social_timestamp,subreddit_almosthomeless,subreddit_anxiety,subreddit_assistance,subreddit_domesticviolence,subreddit_food_pantry,subreddit_homeless,subreddit_ptsd,subreddit_relationships,...,11365,11366,11367,11368,11369,11370,11371,11372,11373,11374
0,0.800000,1521614353,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.000000,1527009817,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.800000,1535935605,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.600000,1516429555,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.800000,1539809005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2833,1.000000,1515187044,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2834,1.000000,1539827412,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2835,1.000000,1539269312,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2836,0.571429,1488938143,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
new_df["confidence"].dtype

dtype('float64')

In [53]:
new_df.dtypes

confidence                  float64
social_timestamp              int64
subreddit_almosthomeless    float64
subreddit_anxiety           float64
subreddit_assistance        float64
                             ...   
11370                       float64
11371                       float64
11372                       float64
11373                       float64
11374                       float64
Length: 11387, dtype: object

In [48]:
Y=df["label"]

In [36]:
Y

,label
0,1
1,0
2,1
3,1
4,1
...,...
2833,0
2834,1
2835,0
2836,0


In [49]:
X_train,X_test,Y_train,Y_test=train_test_split(new_df,Y,test_size=0.2,random_state=42)

In [62]:
X_train.columns=X_train.columns.astype(str)
X_test.columns=X_test.columns.astype(str)

In [64]:
random_classif=RandomForestClassifier(n_estimators)
random_classif.fit(X_train,Y_train)
random_classif.score(X_test,Y_test)

0.7306338028169014

In [69]:
deci_classif=DecisionTreeClassifier()
deci_classif.fit(X_train,Y_train)
deci_classif.score(X_test,Y_test)

0.5757042253521126

In [70]:
grad_classif=GradientBoostingClassifier()
grad_classif.fit(X_train,Y_train)
grad_classif.score(X_test,Y_test)

0.6883802816901409

In [71]:
ada_classif=AdaBoostClassifier()
ada_classif.fit(X_train,Y_train)
ada_classif.score(X_test,Y_test)

0.6795774647887324

In [72]:
bagg_classif=BaggingClassifier()
bagg_classif.fit(X_train,Y_train)
bagg_classif.score(X_test,Y_test)

0.6514084507042254

In [76]:
log_reg=LogisticRegression()
log_reg.fit(X_train,Y_train)
log_reg.score(X_test,Y_test)

0.5369718309859155

In [84]:
X_train.shape

(2270, 11387)

# # Deep learning

In [106]:
from keras.layers import Dense,Input,BatchNormalization,Dropout
from keras.models import Model

In [111]:
inputs=Input(shape=(11387,))
d=Dense(32,activation="relu")(inputs)
d=Dense(64,activation="relu")(d)
d=Dense(32,activation="relu")(d)
b=BatchNormalization()(d)
d=Dense(64,activation="relu")(d)
d=Dense(64,activation="relu")(d)
d=Dropout(0.5)(d)
d=Dense(32,activation="relu")(d)
b=BatchNormalization()(d)

d=Dense(64,activation="relu")(b)
d=Dense(64,activation="relu")(d)
d=Dense(32,activation="relu")(d)
d=Dropout(0.5)(d)

d=Dense(64,activation="relu")(d)
d=Dense(64,activation="relu")(d)
d=Dropout(0.5)(d)

d=Dense(32,activation="relu")(d)
d=Dense(64,activation="relu")(d)
d=Dropout(0.5)(d)

d=Dense(64,activation="relu")(d)
b=BatchNormalization()(d)

d=Dense(32,activation="relu")(b)
d=Dense(64,activation="relu")(d)
d=Dropout(0.5)(d)

d=Dense(64,activation="relu")(d)
d=Dense(32,activation="relu")(d)
d=Dropout(0.5)(d)

d=Dense(64,activation="relu")(d)
d=Dense(64,activation="relu")(d)
b=BatchNormalization()(d)

d=Dense(32,activation="relu")(b)
d=Dense(64,activation="relu")(d)
d=Dense(64,activation="relu")(d)
d=Dropout(0.5)(d)

d=Dense(32,activation="relu")(d)
d=Dense(64,activation="relu")(d)
b=BatchNormalization()(d)

d=Dense(64,activation="relu")(b)
d=Dense(32,activation="relu")(d)
d=Dense(64,activation="relu")(d)
d=Dense(64,activation="relu")(d)
b=BatchNormalization()(d)

d=Dense(32,activation="relu")(b)
d=Dense(64,activation="relu")(d)
d=Dense(64,activation="relu")(d)
d=Dense(32,activation="relu")(d)
d=Dropout(0.5)(d)

d=Dense(64,activation="relu")(d)
b=BatchNormalization()(d)

d=Dense(64,activation="relu")(b)
d=Dense(32,activation="relu")(d)
d=Dense(64,activation="relu")(d)
d=Dense(64,activation="relu")(d)
d=Dense(320,activation="relu")(d)
d=Dense(64,activation="relu")(d)
d=Dense(640,activation="relu")(d)
d=Dense(32,activation="relu")(d)
b=BatchNormalization()(d)

d=Dense(64,activation="relu")(b)
d=Dense(640,activation="relu")(d)
d=Dense(32,activation="relu")(d)
d=Dense(64,activation="relu")(d)
d=Dropout(0.5)(d)

d=Dense(64,activation="relu")(d)
d=Dense(320,activation="relu")(d)
d=Dense(64,activation="relu")(d)
d=Dropout(0.5)(d)

d=Dense(64,activation="relu")(d)
d=Dense(32,activation="relu")(d)
b=BatchNormalization()(d)

d=Dense(640,activation="relu")(b)
d=Dense(640,activation="relu")(d)
d=Dense(320,activation="relu")(d)
d=Dense(640,activation="relu")(d)
d=Dropout(0.5)(d)

d=Dense(640,activation="relu")(d)
d=Dense(320,activation="relu")(d)
b=BatchNormalization()(d)

d=Dense(6400,activation="relu")(b)
d=Dense(6400,activation="relu")(d)
d=Dense(3200,activation="relu")(d)
b=BatchNormalization()(d)

d=Dense(64,activation="relu")(b)
outputs=Dense(1,activation='sigmoid')(d)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])


In [112]:
model.fit(X_train,Y_train,batch_size=32,epochs=5,validation_split=0.2)

Epoch 1/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 61s 693ms/step - accuracy: 0.5451 - loss: 0.9886 - val_accuracy: 0.5022 - val_loss: 0.6975
Epoch 2/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 39s 680ms/step - accuracy: 0.5038 - loss: 0.8214 - val_accuracy: 0.4978 - val_loss: 0.7050
Epoch 3/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 39s 692ms/step - accuracy: 0.4743 - loss: 0.8249 - val_accuracy: 0.4978 - val_loss: 0.6962
Epoch 4/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 39s 686ms/step - accuracy: 0.5125 - loss: 0.7793 - val_accuracy: 0.5022 - val_loss: 0.6954
Epoch 5/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 39s 684ms/step - accuracy: 0.4931 - loss: 0.7595 - val_accuracy: 0.4978 - val_loss: 0.7112


In [113]:
loss,accuracy=model.evaluate(X_test,Y_test)

18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.4560 - loss: 0.7267
